# Trivia quiz using DBpedia

In this exercise, you'll have to answer some trivia questions with the help of DBpedia.  

In [ ]:
pip install ipytest

In [ ]:
from typing import Any, Tuple

import hashlib
import numpy as np
import requests
import urllib.parse

Utility method for fetching all triples associated with a given entity from DBpedia.

In [ ]:
def get_triples(entity_id: str) -> Tuple[str, str, Any]:
    """Returns all triples from DBpedia associated with a given entity.
        
    Args:
        entity_id: Unique entity identifier (e.g., "Kimi_Räikkönen")
    
    Yields:
        (subject, predicate, object) triples where the given entity
        stands either as subject or as object.
        
    Raises:
        ValueError: If invalid or not the canonical entity ID is provided.
    """
    # Fetch DBpedia entry in JSON format.
    # <SPO> triples are represented in the JSON as {S: P: [O1, O2, ...]},
    # i.e., multi-valued predicates are grouped together.
    data = requests.get(f"http://dbpedia.org/data/{entity_id}.json").json()
    
    # Check whether valid entity ID is provided.
    entity_uri = f"http://dbpedia.org/resource/{entity_id}"
    if entity_uri not in data:
        raise ValueError("Invalid entity ID.")
        
    # Check whether this is the canonical entity ID.
    if "http://www.w3.org/1999/02/22-rdf-syntax-ns#type" not in data[entity_uri]:
        raise ValueError("Not the canonical entity ID.")

    for subj, vals in data.items():                
        for pred, objs in vals.items():
            for obj in objs:
                yield subj, pred, obj["value"]

## Example

Compare with the information at https://dbpedia.org/page/Kimi_R%C3%A4ikk%C3%B6nen.

In [ ]:
triples_kimi = [(s, p, o) for s, p, o in get_triples("Kimi_Räikkönen")]

for s, p, o in triples_kimi:
    print(s, p, o)

Types.

In [ ]:
for s, p, o in triples_kimi:
    if s != "http://dbpedia.org/resource/Kimi_Räikkönen":
        continue
    if p == "http://www.w3.org/1999/02/22-rdf-syntax-ns#type":
        print(o)

Birth place and date.

In [ ]:
for s, p, o in triples_kimi:
    if s != "http://dbpedia.org/resource/Kimi_Räikkönen":
        continue
    if "birthPlace" in p or "birthDate" in p:
        print(o)

## Quiz questions

Answer the questions below.

You're allowed to find the exact answers either manually or programmatically, as long as you get the underlying data programmatically using the provided `fetch_entity()` method.

Since tests for this exercise would reveal the correct answer, they are based on MD5 hashes.

### 1) How many Formula 1 Grand Prixes has Kimi Räikkönen won?

In [ ]:
wins = [s for s, p, o in triples_kimi if "ontology/firstDriver" in p and "Grand_Prix" in s]
num_wins = len(wins)

In [ ]:
assert hashlib.md5(np.int8(num_wins)).hexdigest() == "f5a7e477cd3042b49a9085d62307cd28"

### 2) Is Kimi Räikkönen married?

Return the string "Y" or "N".

In [ ]:
married = "N"
for s, p, o in triples_kimi:
    if "ontology/spouse" in p:
        married = "Y"
        break

In [ ]:
assert hashlib.md5(bytes(married, "utf-8")).hexdigest() == "57cec4137b614c87cb4e24a3d003a3e0"

### 3) In which country did Kimi win his first Grand Prix?

Return the canonical name of the country as a string.

Utility functions to help answer this question.

In [ ]:
def get_entity_id(uri: str) -> str:
    """Return the entity_id part of an URI (part after the last slash)."""
    return uri.split("/")[-1]

def is_country(uri: str) -> bool:
    """Determines if a given URI is a country."""
    for _, p, o in get_triples(get_entity_id(uri)):
        if p != "http://www.w3.org/1999/02/22-rdf-syntax-ns#type":
            continue
        if o == "http://dbpedia.org/ontology/Country":
            return True
    return False

In [ ]:
# Get first win.
first_win = None
for s, p, o in triples_kimi:
    if "ontology/firstWin" in p:
        first_win = get_entity_id(o)  # Keep only entity_id, not the whole URI.
        break

# Determine the location of that GP.
country = None
for _, p, o in get_triples(first_win):
    if "ontology/location" in p:
        if is_country(o):
            country = get_entity_id(o)

In [ ]:
assert hashlib.md5(bytes(country, "utf-8")).hexdigest() == "3f0e49c46cbde0c7adf5ea04a97ab261"